In [3]:
pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)

[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
import cv2
import mediapipe as mp
import pandas as pd
from tqdm import tqdm

# Rutas
INPUT_DIR = '../../data/dataset_raw'  # Carpeta con videos
OUTPUT_DIR = 'dataset_landmarks'  # Carpeta donde se guardarán los CSV

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Inicializar MediaPipe
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False)
mp_drawing = mp.solutions.drawing_utils

# Función para extraer landmarks de un solo frame
def extract_landmarks(results):
    if not results.pose_landmarks:
        return None
    row = []
    for lm in results.pose_landmarks.landmark:
        row.extend([lm.x, lm.y, lm.z, lm.visibility])
    return row

# Procesar todos los videos
for file in tqdm(os.listdir(INPUT_DIR)):
    if not file.endswith(".mp4"):
        continue

    video_path = os.path.join(INPUT_DIR, file)
    cap = cv2.VideoCapture(video_path)

    filename = os.path.splitext(file)[0]
    label = "_".join(filename.split("_")[2:4])  # e.g., "caminar_hacia"

    landmarks_list = []
    frame_id = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)

        landmarks = extract_landmarks(results)
        if landmarks:
            landmarks_list.append([frame_id, label] + landmarks)

        frame_id += 1

    cap.release()

    # Guardar en CSV
    if landmarks_list:
        df = pd.DataFrame(landmarks_list)
        num_landmarks = int((len(df.columns) - 2) / 4)
        col_names = ["frame", "label"] + [
            f"{joint}_{coord}"
            for joint in range(num_landmarks)
            for coord in ['x', 'y', 'z', 'vis']
        ]
        df.columns = col_names
        df.to_csv(os.path.join(OUTPUT_DIR, filename + ".csv"), index=False)


I0000 00:00:1747527551.740920  471044 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M2
  0%|          | 0/165 [00:00<?, ?it/s]W0000 00:00:1747527551.832480  478305 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747527551.844370  478305 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747527551.859293  478303 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
100%|██████████| 165/165 [04:33<00:00,  1.66s/it]
